In [2]:
#firstly,I need to fire up the Graphlab
from __future__ import division
import graphlab
import math
import string

In [3]:
#extract data from my file
products = graphlab.SFrame('amazon_baby.gl/')

This non-commercial license of GraphLab Create for academic use is assigned to 821691892@qq.com and will expire on July 05, 2017.


[INFO] graphlab.cython.cy_server: GraphLab Create v1.10.1 started. Logging: C:\Users\lixx3527\AppData\Local\Temp\graphlab_server_1473265646.log.0


In [4]:
#a quick look what the file looks like
products

name,review,rating
Planetwise Flannel Wipes,"These flannel wipes areOK, but in my opinion ...",3.0
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0


In [5]:
#let's explore the products[269]
products[269]

{'name': 'The First Years Massaging Action Teether',
 'rating': 5.0,
 'review': 'A favorite in our house!'}

In [6]:
#defile a function which is used to count words in the review coloumn
def remove_punctuation(text):
    import string
    return text.translate(None, string.punctuation) 

review_without_puctuation = products['review'].apply(remove_punctuation)
products['word_count'] = graphlab.text_analytics.count_words(review_without_puctuation)

In [7]:
#let's explore the the word_count in products[269]
products[269]['word_count']

{'a': 1L, 'favorite': 1L, 'house': 1L, 'in': 1L, 'our': 1L}

In [8]:
#We will ignore all reviews with rating = 3, since they tend to have a neutral sentiment.
products = products[products['rating'] != 3]
len(products)

166752

In [9]:
#now, we just use lambda functions to assign sentiment value +1 or -1 to every member in the list
products['sentiment'] = products['rating'].apply(lambda rating : +1 if rating > 3 else -1)
products

name,review,rating,word_count,sentiment
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,"{'and': 3L, 'love': 1L,'it': 3L, 'highly': 1L, ...",1
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,"{'and': 2L, 'quilt': 1L,'it': 1L, 'comfortable': ...",1
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0,"{'and': 3L, 'ingenious':1L, 'love': 2L, 'is': ...",1
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,"{'and': 2L, 'all': 2L,'help': 1L, 'cried': 1L, ...",1
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0,"{'and': 2L, 'cute': 1L,'help': 2L, 'habit': 1L, ...",1
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0,"{'shop': 1L, 'be': 1L,'is': 1L, 'bound': 1L, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0,"{'and': 2L, 'all': 1L,'right': 1L, 'able': 1L, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0,"{'and': 1L, 'fantastic':1L, 'help': 1L, 'give': ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,"{'all': 1L, 'standarad':1L, 'another': 1L, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",I love this journal andour nanny uses it ...,4.0,"{'all': 2L, 'nannys': 1L,'just': 1L, 'sleep': 2L, ...",1


In [10]:
#split the data into two parts
train_data, test_data = products.random_split(.8, seed=1)
print len(train_data)
print len(test_data)

133416
33336


In [11]:
#Then, we need to create a logistics model using Graphlab functions
sentiment_model = graphlab.logistic_classifier.create(train_data,
                                                      target = 'sentiment',
                                                      features=['word_count'],
                                                      validation_set=None)

Logistic regression:

--------------------------------------------------------

Number of examples          : 133416

Number of classes           : 2

Number of feature columns   : 1

...

| 5         | 12       | 3.000000  | 5.823412     | 0.960964          |

| 6         | 13       | 3.000000  | 6.307357     | 0.975033          |

+-----------+----------+-----------+--------------+-------------------+

TERMINATED: Terminated due to numerical difficulties.

This model may not be ideal. To improve it, consider doing one of the following:
(a) Increasing the regularization.
(b) Standardizing the input data.
(c) Removing highly correlated features.
(d) Removing `inf` and `NaN` values in the training data.

In [12]:
sentiment_model

Class                         : LogisticClassifier

Schema
------
Number of coefficients        : 121713
Number of examples            : 133416
Number of classes             : 2
Number of feature columns     : 1
Number of unpacked features   : 121712

Hyperparameters
---------------
L1 penalty                    : 0.0
L2 penalty                    : 0.01

Training Summary
----------------
Solver                        : lbfgs
Solver iterations             : 6
Solver status                 : TERMINATED: Terminated due to numerical difficulties.
Training time (sec)           : 6.7902

Settings
--------
Log-likelihood                : inf

Highest Positive Coefficients
-----------------------------
word_count[mobileupdate]      : 41.9847
word_count[placeid]           : 41.7354
word_count[labelbox]          : 41.151
word_count[httpwwwamazoncomreviewrhgg6qp7tdnhbrefcmcrprcmtieutf8asinb00318cla0nodeid]: 40.0454
word_count[knobskeeping]      : 36.2091

Lowest Negative Coefficients
-----------

In [13]:
weights = sentiment_model.coefficients
weights.column_names()

['name', 'index', 'class', 'value', 'stderr']

In [18]:
#count how many weights do they have positive rating values
num_positive_weights = sum(weights['value']>=0)
num_negative_weights = sum(weights['value']<0)

print "Number of positive weights: %s " % num_positive_weights
print "Number of negative weights: %s " % num_negative_weights

Number of positive weights: 68419 
Number of negative weights: 53294 


In [19]:
sample_test_data = test_data[10:13]
print sample_test_data['rating']
sample_test_data

[5.0, 2.0, 1.0]


name,review,rating,word_count,sentiment
Our Baby Girl Memory Book,Absolutely love it andall of the Scripture in ...,5.0,"{'and': 2L, 'all': 1L,'love': 1L, ...",1
Wall Decor RemovableDecal Sticker - Colorful ...,Would not purchase againor recommend. The decals ...,2.0,"{'and': 1L, 'wall': 1L,'them': 1L, 'decals': ...",-1
New Style Trailing CherryBlossom Tree Decal ...,Was so excited to getthis product for my baby ...,1.0,"{'all': 1L, 'money': 1L,'into': 1L, 'it': 3L, ...",-1


In [20]:
sample_test_data[0]['review']

'Absolutely love it and all of the Scripture in it.  I purchased the Baby Boy version for my grandson when he was born and my daughter-in-law was thrilled to receive the same book again.'

In [21]:
sample_test_data[1]['review']

'Would not purchase again or recommend. The decals were thick almost plastic like and were coming off the wall as I was applying them! The would NOT stick! Literally stayed stuck for about 5 minutes then started peeling off.'

In [22]:
#use the model to predict 
scores = sentiment_model.predict(sample_test_data, output_type='margin')
print scores

[6.734619727059735, -5.734130996760568, -14.668460404469037]


In [24]:
print "Class predictions according to GraphLab Create:" 
print sentiment_model.predict(sample_test_data)

Class predictions according to GraphLab Create:
[1L, -1L, -1L]


In [26]:
#predict the probabilities of the positive rating
print "Class predictions according to GraphLab Create:" 
print sentiment_model.predict(sample_test_data, output_type='probability')

Class predictions according to GraphLab Create:
[0.9988123848377203, 0.003223268181799683, 4.261557996653207e-07]


In [28]:
#then we predict the probability of positive rating for every toy in the list
#and we also need to the most positive rating in the test_data
predictions=sentiment_model.predict(test_data,output_type='probability')
test_data['probability']=predictions
test_data.topk('probability',20).print_rows(num_rows=20)

+-------------------------------+-------------------------------+--------+
|              name             |             review            | rating |
+-------------------------------+-------------------------------+--------+
| Britax Decathlon Convertib... | I researched a few differe... |  4.0   |
| Ameda Purely Yours Breast ... | As with many new moms, I c... |  4.0   |
| Traveling Toddler Car Seat... | I am sure this product wor... |  2.0   |
| Shermag Glider Rocker Comb... | After searching in stores ... |  4.0   |
| Cloud b Sound Machine Soot... | First off, I love plush sh... |  5.0   |
| JP Lizzy Chocolate Ice Cla... | I got this bag as a presen... |  4.0   |
| Fisher-Price Rainforest Me... | My daughter wasn't able to... |  5.0   |
| Lilly Gold Sit 'n' Stroll ... | I just completed a two-mon... |  5.0   |
|  Fisher-Price Deluxe Jumperoo | I had already decided that... |  5.0   |
| North States Supergate Pre... | I got a couple of these to... |  4.0   |
|   Munchkin Mozart Magic

In [30]:
#then find the 20 most least positive ratings' subjects in the test_data
test_data.sort('probability').print_rows(num_rows=20)


+-------------------------------+-------------------------------+--------+
|              name             |             review            | rating |
+-------------------------------+-------------------------------+--------+
| Jolly Jumper Arctic Sneak ... | I am a "research-aholic" i... |  5.0   |
| Levana Safe N'See Digital ... | This is the first review I... |  1.0   |
| Snuza Portable Baby Moveme... | I would have given the pro... |  1.0   |
| Fisher-Price Ocean Wonders... | We have not had ANY luck w... |  2.0   |
| VTech Communications Safe ... | This is my second video mo... |  1.0   |
| Safety 1st High-Def Digita... | We bought this baby monito... |  1.0   |
| Chicco Cortina KeyFit 30 T... | My wife and I have used th... |  1.0   |
| Prince Lionheart Warmies W... | *****IMPORTANT UPDATE*****... |  1.0   |
| Valco Baby Tri-mode Twin S... | I give one star to the dim... |  1.0   |
| Adiri BPA Free Natural Nur... | I will try to write an obj... |  2.0   |
| Munchkin Nursery Projec

In [35]:
#compute the accuracy of the model
def get_classification_accuracy(model, data, true_labels):
    # First get the predictions
    predictions=model.predict(data)
    
    # Compute the number of correctly classified examples
    count=0
    for i in range(len(predictions)):
        if predictions[i]==true_labels[i]:
            count+=1
        
    accuracy=float(count)/len(predictions)
    return accuracy

In [34]:
#get the accuracy of predictions of the model(in terms of test_data)
get_classification_accuracy(sentiment_model, test_data, test_data['sentiment'])

0.9145368370530358

In [36]:
#right now, we are going to reduce the number of words we need to train in our new model
significant_words = ['love', 'great', 'easy', 'old', 'little', 'perfect', 'loves', 
      'well', 'able', 'car', 'broke', 'less', 'even', 'waste', 'disappointed', 
      'work', 'product', 'money', 'would', 'return']

In [37]:
len(significant_words)

20

In [38]:
train_data['word_count_subset'] = train_data['word_count'].dict_trim_by_keys(significant_words, exclude=False)
test_data['word_count_subset'] = test_data['word_count'].dict_trim_by_keys(significant_words, exclude=False)

In [39]:
train_data[0]['review']

'it came early and was not disappointed. i love planet wise bags and now my wipe holder. it keps my osocozy wipes moist and does not leak. highly recommend it.'

In [40]:
print train_data[0]['word_count_subset']

{'love': 1L, 'disappointed': 1L}


In [41]:
simple_model = graphlab.logistic_classifier.create(train_data,
                                                   target = 'sentiment',
                                                   features=['word_count_subset'],
                                                   validation_set=None)
simple_model

Logistic regression:

--------------------------------------------------------

Number of examples          : 133416

Number of classes           : 2

Number of feature columns   : 1

...

| 5         | 6        | 0.764189     | 0.866815          |

| 6         | 7        | 0.912529     | 0.866815          |

+-----------+----------+--------------+-------------------+

SUCCESS: Optimal solution found.

Class                         : LogisticClassifier

Schema
------
Number of coefficients        : 21
Number of examples            : 133416
Number of classes             : 2
Number of feature columns     : 1
Number of unpacked features   : 20

Hyperparameters
---------------
L1 penalty                    : 0.0
L2 penalty                    : 0.01

Training Summary
----------------
Solver                        : newton
Solver iterations             : 6
Solver status                 : SUCCESS: Optimal solution found.
Training time (sec)           : 0.9481

Settings
--------
Log-likelihood                : 44323.7254

Highest Positive Coefficients
-----------------------------
word_count_subset[loves]      : 1.6773
word_count_subset[perfect]    : 1.5145
word_count_subset[love]       : 1.3654
(intercept)                   : 1.2995
word_count_subset[easy]       : 1.1937

Lowest Negative Coefficients
----------------------------
word_count_subset[disappointed]: -2.3551
word_count_subset[ret

In [42]:
#now ,compute new model's predictions accuracy
get_classification_accuracy(simple_model, test_data, test_data['sentiment'])

0.8693004559635229

In [43]:
simple_model.coefficients

name,index,class,value,stderr
(intercept),None,1,1.2995449552,0.0120888541331
word_count_subset,disappointed,1,-2.35509250061,0.0504149888557
word_count_subset,love,1,1.36543549368,0.0303546295109
word_count_subset,well,1,0.504256746398,0.021381300631
word_count_subset,product,1,-0.320555492996,0.0154311321362
word_count_subset,loves,1,1.67727145556,0.0482328275384
word_count_subset,little,1,0.520628636025,0.0214691475665
word_count_subset,work,1,-0.621700012425,0.0230330597946
word_count_subset,easy,1,1.19366189833,0.029288869202
word_count_subset,great,1,0.94469126948,0.0209509926591


In [44]:
simple_model.coefficients.sort('value', ascending=False).print_rows(num_rows=21)

+-------------------+--------------+-------+-----------------+-----------------+
|        name       |    index     | class |      value      |      stderr     |
+-------------------+--------------+-------+-----------------+-----------------+
| word_count_subset |    loves     |   1   |  1.67727145556  | 0.0482328275384 |
| word_count_subset |   perfect    |   1   |  1.51448626703  |  0.049861952294 |
| word_count_subset |     love     |   1   |  1.36543549368  | 0.0303546295109 |
|    (intercept)    |     None     |   1   |   1.2995449552  | 0.0120888541331 |
| word_count_subset |     easy     |   1   |  1.19366189833  |  0.029288869202 |
| word_count_subset |    great     |   1   |  0.94469126948  | 0.0209509926591 |
| word_count_subset |    little    |   1   |  0.520628636025 | 0.0214691475665 |
| word_count_subset |     well     |   1   |  0.504256746398 |  0.021381300631 |
| word_count_subset |     able     |   1   |  0.191438302295 | 0.0337581955697 |
| word_count_subset |     ol

In [56]:
#get the value of coefficients [significant_words] using sentiment_model
copy=sentiment_model.coefficients
check=[]
for ele in copy['index']:
    if ele in significant_words:
        check.append(True)
    else:
        check.append(False)
copy['tf']=check
sentiment_model_subset=copy[copy['tf']==True]
sentiment_model_subset.sort('value',ascending=False).print_rows(num_rows=20)

+------------+--------------+-------+------------------+--------+----+
|    name    |    index     | class |      value       | stderr | tf |
+------------+--------------+-------+------------------+--------+----+
| word_count |   perfect    |   1   |  1.75190114392   |  None  | 1  |
| word_count |    loves     |   1   |   1.5664851757   |  None  | 1  |
| word_count |     love     |   1   |  1.43301685439   |  None  | 1  |
| word_count |    great     |   1   |  1.31459245039   |  None  | 1  |
| word_count |     easy     |   1   |  1.21346937822   |  None  | 1  |
| word_count |    little    |   1   |  0.674162457499  |  None  | 1  |
| word_count |     well     |   1   |  0.627964877567  |  None  | 1  |
| word_count |     car      |   1   |  0.195263670618  |  None  | 1  |
| word_count |     able     |   1   |  0.174331272552  |  None  | 1  |
| word_count |     old      |   1   | 0.00912230113669 |  None  | 1  |
| word_count |   product    |   1   | -0.0430899241758 |  None  | 1  |
| word

In [57]:
#First, compute the classification accuracy of the sentiment_model on the train_data:
get_classification_accuracy(sentiment_model, train_data, train_data['sentiment'])

0.979440247046831

In [58]:
#Now, compute the classification accuracy of the simple_model on the train_data:
get_classification_accuracy(simple_model, train_data, train_data['sentiment'])

0.8668150746537147

In [61]:
graphlab.version

'1.10.1'